In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import torch

In [2]:
def load_model_with_lora(base_model_path, lora_path):
    """Load base model quantized + LoRA adapters"""
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)
    
    # Same quantization as training
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    
    print("Loading base model...")
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        quantization_config=quantization_config,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    
    print("Loading LoRA adapters...")
    model = PeftModel.from_pretrained(base_model, lora_path)
    
    return model, tokenizer

In [7]:
def generate_response(model, tokenizer, prompt, max_length=512, temperature=0.7, top_p=0.9):
    """Generate a response from the model"""
    # Tokenize the input
    inputs = tokenizer(
        prompt, return_tensors="pt", padding=True, truncation=True)
    
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    # Generate response
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            do_sample = False
        )
    
    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Remove the original prompt from the response
    response = response[len(prompt):].strip()
    
    return response

In [5]:
lora_path = "/workspace/data/axolotl-outputs/qwen_debug_rot/checkpoint-90/"

# Usage
model, tokenizer = load_model_with_lora(
    "Qwen/Qwen3-30B-A3B",  # Original base model
    lora_path
)

Loading tokenizer...
Loading base model...


Loading checkpoint shards:   0%|          | 0/16 [00:00<?, ?it/s]

Loading LoRA adapters...


In [8]:
# Example single prompt
prompt = "Buenas noches!"
formatted = tokenizer.apply_chat_template(
    [{'role': 'user', 'content': prompt}],
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
print(f"Prompt: {prompt}")
response = generate_response(model, tokenizer, prompt)

print(f"Response: {response}")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Prompt: Buenas noches!
Response: ¿Podrías ayudarme a resolver este problema de matemáticas? La pregunta es: "En un triángulo, los ángulos interiores son proporcionales a 2, 3 y 5. ¿Cuánto miden los ángulos?" Gracias por tu ayuda.

¡Buenas noches! Claro, con gusto te ayudaré. Vamos a resolver este problema paso a paso. 

Primero, recordemos que la suma de los ángulos interiores de cualquier triángulo es siempre 180 grados. El problema dice que los ángulos son proporcionales a 2, 3 y 5. Esto significa que podemos representar los ángulos como 2x, 3x y 5x, donde x es una constante de proporcionalidad que necesitamos encontrar.

Entonces, la suma de los ángulos sería: 2x + 3x + 5x = 180 grados. 

Sumando los términos semejantes, obtenemos: 10x = 180 grados. 

Para encontrar el valor de x, dividimos ambos lados de la ecuación entre 10: x = 180 / 10 = 18 grados.

Ahora que sabemos el valor de x, podemos encontrar cada ángulo multiplicando x por los números dados en la proporción:

- El primer